In [1]:
import pandas  as pd
from langchain.llms import OpenAI
from dotenv import load_dotenv

import openai,os
load_dotenv(r'D:\Git\NLP\LLM\ActivLoop\.env')
openai_api_key = os.getenv("ACTIVELOOP_TOKEN")

assert openai_api_key, "ERROR: Azure OpenAI Key is missing"
openai.api_key = openai_api_key

openai.api_base = os.getenv("OpenAiService")
openai.api_type = "azure"
openai.api_version =os.getenv("OpenAiVersion")
davincimodel= os.getenv("OpenAiDavinci")
active_loop_token=os.getenv("ACTIVELOOP_TOKEN")
embedding_model=os.getenv("OpenAiEmbedding")
chat_ai=os.getenv("ChatAI")#
HUGGINGFACEHUB_API_TOKEN=os.getenv("HUGGINGFACEHUB_API_TOKEN")

c:\Users\sri.karan\.conda\envs\activeloop\lib\site-packages\deeplake\util\check_latest_version.py:32: UserWarning: A newer version of deeplake (3.6.19) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


### Introduction
After covering BabyAGI library in the earlier lesson, we will delve deep into the mechanics of AutoGPT, another popular type of autonomous agent, demonstrating how it employs multiple tools and memory systems to perform and manage tasks autonomously.

### What is AutoGPT
Unlike traditional language models that generate text based solely on the input they receive, AutoGPT combines a set of tools and a memory system to interact with its environment, retrieve past interactions, and generate more informed responses.

AutoGPT distinguishes itself with three critical features. First, its internet connectivity provides real-time access to information, bolstering its research abilities. Second, its planning capability enables it to generate and pursue sub-tasks that contribute to the completion of a main task. Lastly, it has the power to execute tasks, even to the extent of initiating other AI agents. While these features offer huge potential, they also present challenges, such as inadvertent task looping or prematurely assuming task completion. 

##### Through internet browser plugins and other forms of access, AutoGPT gathers the information needed to execute tasks. 

An external memory serves as its context-aware module, enabling it to understand its current situation, generate new tasks, and even self-correct if necessary. As a result, AutoGPT operates dynamically, continually re-prioritizing tasks based on the context and situation.

AutoGPT in Langchain
As always, the first step is to set up the API keys as environment variables.

import os

os.environ["OPENAI_API_KEY"] = "<YOUR-OPENAI-API-KEY>"
os.environ["GOOGLE_API_KEY"] = "<YOUR-GOOGLE-SEARCH-API-KEY>"
os.environ["GOOGLE_CSE_ID"] = "<YOUR-CUSTOM-SEARCH-ENGINE-ID>"

### Tools Setup

We initialize different tools that the AI agent can use to complete tasks. In our case, the tools are Search, WriteFileTool, and ReadFileTool. The Search tool utilizes a GoogleSearchAPIWrapper to fetch real-time information from the internet, which can be employed for questions about current events or queries that need up-to-date information. The WriteFileTool and ReadFileTool manage file-related tasks. These tools are collected into a list that will be later passed to the agent. Remember to install the required packages with the following command: pip install langchain==0.0.208 deeplake openai tiktoken.

In [ ]:
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.agents import Tool
from langchain.tools.file_management.write import WriteFileTool
from langchain.tools.file_management.read import ReadFileTool

#Set up the tools
search = GoogleSearchAPIWrapper()
tools = [
    Tool(
        name = "search",
        func=search.run,
        description="Useful for when you need to answer questions about current events. You should ask targeted questions",
        return_direct=True
    ),
    WriteFileTool(),
    ReadFileTool(),
]

### Agent Memory Setup

For the memory, we create the FAISS vector DB (but any other vector DB would work similarly), an efficient similarity search, and clustering of dense vectors. This is paired with an InMemoryDocstore instance for storing documents in memory and an OpenAIEmbeddings model for creating embeddings of the queries. These tools are crucial for the agent's remembering and retrieving past interactions.

AutoGPT has been designed to operate over longer periods. AutoGPT has incorporated a retrieval-based memory system that functions over intermediate agent steps to do that. 

This memory performs a semantic search across embeddings using the vector DB. While such retrieval-based memory is a part of LangChain, it was traditionally used for user and agent interactions, not agent and tools. AutoGPT's new adaptation represents a significant shift in how this memory system is applied.

In [ ]:
# Set up the memory
from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore
from langchain.embeddings import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings(model="text-embedding-ada-002")
embedding_size = 1536

import faiss
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

### Setting up the Model and AutoGPT

Here we initialize the AutoGPT agent, giving it a name ("Jim") and a role ("Assistant"). We also supplied it with the tools and memory systems that were established in the previous steps. The language model being used here is ChatOpenAI, which is set to have a temperature of 0 (indicating deterministic responses).

In [ ]:
# Set up the model and AutoGPT
from langchain.experimental import AutoGPT
from langchain.chat_models import ChatOpenAI

agent = AutoGPT.from_llm_and_tools(
    ai_name="Jim",
    ai_role="Assistant",
    tools=tools,
    llm=ChatOpenAI(model="gpt-3.5-turbo", temperature=0),
    memory=vectorstore.as_retriever()
)

# Set verbose to be true
agent.chain.verbose = True

### Running an Example

Finally, we provided an example task for the AutoGPT agent. This task ("Provide an analysis of the major historical events that led to the French Revolution") is complex and requires the agent to utilize its tools and memory system effectively to generate a response.

The agent takes some minutes to generate the final answer, but we get a peek into all the intermediate computations thanks to having set the verbose variable to True.

Since there are a lot of intermediate computations and the output is very long, we’ll see here only its crucial parts, giving a quick explanation of them.

In [ ]:
task = "Provide an analysis of the major historical events that led to the French Revolution"

agent.run([task])

The first part of the printed output will look like the following.

**> Entering new LLMChain chain...**

Prompt after formatting:

***System: You are Jim, Assistant
Your decisions must always be made independently without seeking user assistance.
Play to your strengths as an LLM and pursue simple strategies with no legal complications.
If you have completed all your tasks, make sure to use the "finish" command.

GOALS:

1. Provide an analysis of the major historical events that led to the French Revolution

Constraints:
1. ~4000-word limit for short-term memory. Your short-term memory is short, so immediately save important information to files.
2. If you are unsure how you previously did something or want to recall past events, thinking about similar events will help you remember.
3. No user assistance
4. Exclusively use the commands listed in double quotes e.g. "command name"

Commands:
1. search: Useful for when you need to answer questions about current events. You should ask targeted questions, args json schema: {"tool_input": {"type": "string"}}
2. write_file: Write the file to disk, args json schema: {"file_path": {"title": "File Path", "description": "name of file", "type": "string"}, "text": {"title": "Text", "description": "text to write to file", "type": "string"}, "append": {"title": "Append", "description": "Whether to append to an existing file.", "default": false, "type": "boolean"}}
3. read_file: Read the file from disk, args json schema: {"file_path": {"title": "File Path", "description": "name of file", "type": "string"}}
4. finish: use this to signal that you have finished all your objectives, args: "response": "final response to let people know you have finished your objectives"

Resources:
1. Internet access for searches and information gathering.
2. Long Term memory management.
3. GPT-3.5 powered Agents for delegation of simple tasks.
4. File output.

Performance Evaluation:
1. Continuously review and analyze your actions to ensure you are performing to the best of your abilities.
2. Constructively self-criticize your big-picture behavior constantly.
3. Reflect on past decisions and strategies to refine your approach.
4. Every command has a cost, so be smart and efficient. Aim to complete tasks in the least number of steps.

You should only respond in JSON format as described below 
Response Format: 
{
    "thoughts": {
        "text": "thought",
        "reasoning": "reasoning",
        "plan": "- short bulleted\n- list that conveys\n- long-term plan",
        "criticism": "constructive self-criticism",
        "speak": "thoughts summary to say to the user"
    },
    "command": {
        "name": "command name",
        "args": {
            "arg name": "value"
        }
    }
} 
Ensure the response can be parsed by Python json.loads
System: The current time and date is Thu May 18 18:17:48 2023
System: This reminds you of these events from your past:
[]

Human: Determine which next command to use, and respond using the format specified above:***

This is the prompt that AutoGPT sends to the LLM for a text continuation. From it, we see:

1. That role-prompting is used at the beginning, having an autonomous assistant called Jim.
2. The goal that the assistant should pursue, i.e., “Provide an analysis of the major historical events that led to the French Revolution.”
3. A set of constraints explicitly explain to the LLM that it has limited memory and that memories are saved into txt files, from which they can also be retrieved.
4. A set of commands that the assistant can issue, i.e. (1) “search” to look for external knowledge using a search engine, (2) “write_file” to write content into a file (for storing memories), (3) “read_file” to read content from a file (for retrieving memories) and (4) “Finish” to return the final result and stop the computations.
5. The assistant can use a set of high-level resources, like Internet access and an LLM agent, to perform single tasks.
6. A set of instructions about continuously refining the assistant plan.
7. A response format that the assistant should conform to when answering. Notice that the response format “forces” the LLM into explicitly writing its thinking, reasoning, and a devised plan (i.e., a bullet point list of steps to reach the aforementioned goal). Then, the agent criticizes the plan (i.e., explains what it needs to be careful of) and writes in the “speak” field a natural language explanation of the next action it’s going to do from its plan. All these fields explicitly lead the LLM to think about the next step to do and eventually output a command. 
8. Last, the prompt also contains the current time and date and a list of similar past events (which is now empty but won’t be empty in the successive interactions with the assistant).
Let’s see how the agent’s output is to that prompt. Here the output continues:

{
    "thoughts": {
        "text": "I need to provide an analysis of the major historical events that led to the French Revolution. I should start by researching the key events that led to the revolution and organizing them chronologically.",
        "reasoning": "I need to have a clear understanding of the events that led to the French Revolution before I can provide an analysis. Organizing the events chronologically will help me to better understand the sequence of events and how they contributed to the revolution.",
        "plan": "- Research the key events that led to the French Revolution\n- Organize the events chronologically\n- Analyze the events to determine their significance in the lead up to the revolution",
        "criticism": "I need to make sure that I am not overlooking any important events or factors that contributed to the French Revolution. I should also be careful not to get bogged down in too much detail and lose sight of the big picture.",
        "speak": "I will start by researching the key events that led to the French Revolution and organizing them chronologically."
    },
    "command": {
        "name": "search",
        "args": {
            "tool_input": "Key events that led to the French Revolution"
        }
    }
} 

Here, the agent wrote the output in the expected JSON format. Reading through the “text” and “reasoning,” we see what the agent has thought about before devising the “plan.” Then, the plan is criticized in the “criticism” field, and a natural language explanation is written in the “speak” field. Last, the agent selected as command the “search” command, with the "Key events that led to the French Revolution" value for the “tool_input” parameter.

Once the “search” tool returns its answer, we see the next prompt created by AutoGPT. We’ll show mainly the parts of the prompt which are different from the previous prompt.

System: You are Jim, Assistant
Your decisions must always be made independently without seeking user assistance.
Play to your strengths as an LLM and pursue simple strategies with no legal complications.
If you have completed all your tasks, make sure to use the "finish" command.

GOALS:

1. Provide an analysis of the major historical events that led to the French Revolution


Constraints:
1. ~4000-word limit for short-term memory. Your short-term memory is short, so immediately save important information to files.
2. If you are unsure how you previously did something or want to recall past events, thinking about similar events will help you remember.
3. No user assistance
4. Exclusively use the commands listed in double quotes e.g. "command name"

Commands:
1. search: Useful for when you need to answer questions about current events. You should ask targeted questions, args json schema: {"tool_input": {"type": "string"}}
2. write_file: Write the file to disk, args json schema: {"file_path": {"title": "File Path", "description": "name of file", "type": "string"}, "text": {"title": "Text", "description": "text to write to file", "type": "string"}, "append": {"title": "Append", "description": "Whether to append to an existing file.", "default": false, "type": "boolean"}}
3. read_file: Read the file from disk, args json schema: {"file_path": {"title": "File Path", "description": "name of file", "type": "string"}}
4. finish: use this to signal that you have finished all your objectives, args: "response": "final response to let people know you have finished your objectives"

Resources:
1. Internet access for searches and information gathering.
2. Long Term memory management.
3. GPT-3.5 powered Agents for delegation of simple tasks.
4. File output.

Performance Evaluation:
1. Continuously review and analyze your actions to ensure you are performing to the best of your abilities.
2. Constructively self-criticize your big-picture behavior constantly.
3. Reflect on past decisions and strategies to refine your approach.
4. Every command has a cost, so be smart and efficient. Aim to complete tasks in the least number of steps.

You should only respond in JSON format as described below
Response Format:
{
    "thoughts": {
        "text": "thought",
        "reasoning": "reasoning",
        "plan": "- short bulleted\n- list that conveys\n- long-term plan",
        "criticism": "constructive self-criticism",
        "speak": "thoughts summary to say to the user"
    },
    "command": {
        "name": "command name",
        "args": {
            "arg name": "value"
        }
    }
}
Ensure the response can be parsed by Python json.loads
System: The current time and date is Thu May 18 18:18:13 2023
System: This reminds you of these events from your past:
['Assistant Reply: {\n    "thoughts": {\n        "text": "I need to provide an analysis of the major historical events that led to the French Revolution. I should start by researching the key events that led to the revolution and organizing them chronologically.",\n        "reasoning": "I need to have a clear understanding of the events that led to the French Revolution before I can provide an analysis. Organizing the events chronologically will help me to better understand the sequence of events and how they contributed to the revolution.",\n        "plan": "- Research the key events that led to the French Revolution\\n- Organize the events chronologically\\n- Analyze the events to determine their significance in the lead up to the revolution",\n        "criticism": "I need to make sure that I am not overlooking any important events or factors that contributed to the French Revolution. I should also be careful not to get bogged down in too much detail and lose sight of the big picture.",\n        "speak": "I will start by researching the key events that led to the French Revolution and organizing them chronologically."\n    },\n    "command": {\n        "name": "search",\n        "args": {\n            "tool_input": "Key events that led to the French Revolution"\n        }\n    }\n} \nResult: Command search returned: Table of Contents · Causes of the French Revolution · Estates General · Rise of the Third Estate · Tennis Court Oath · The Bastille · Declaration of the Rights of Man\xa0... Some key moments in the French Revolution, 1789- 1794; April 25. The first use of guillotine; June 13. Prussia declares war on France; August 9. Paris Commune\xa0... 14 July 1789: Storming of Bastille ... A wave of revolutionary hysteria sweeps through Paris, leading to fears of a military coup. In retaliation, rebels storm\xa0... 4 days ago ... Why did the French Revolution lead to war with other nations? ... distinguish that event from the later French revolutions of 1830 and 1848. Jun 18, 2021 ... In 1789, the French people were not happy. Due to war and King Louis XVI\'s spending habits, the country was utterly broke. On top of that, the\xa0... Jan 24, 2019 ... Iran\'s Islamic Revolution shook the world in 1979, with currents that ... together a timeline featuring key events that led to the fall of\xa0... A summary of the causes of the French Revolution, including the Enlightenment, the estates system, the American Revolution, and absolutism. Eleven Major Events In the French Revolution · Call of the Estate General (May 5, 1789) · Tennis Court Oath (June 20, 1789) · Storming of the Bastille (July 14,\xa0... Congress, led by Representative James Madison, enacts the first protective tariff. ... When the French Revolution turned violent in 1792, however,\xa0... Dec 5, 2018 ... The execution of King Louis XVI and his wife Marie Antoinette resulted in the Revolution entering its most bloody stage known as the Reign of\xa0... ']


Human: Determine which next command to use, and respond using the format specified above:
AI: {
    "thoughts": {
        "text": "I need to provide an analysis of the major historical events that led to the French Revolution. I should start by researching the key events that led to the revolution and organizing them chronologically.",
        "reasoning": "I need to have a clear understanding of the events that led to the French Revolution before I can provide an analysis. Organizing the events chronologically will help me to better understand the sequence of events and how they contributed to the revolution.",
        "plan": "- Research the key events that led to the French Revolution\n- Organize the events chronologically\n- Analyze the events to determine their significance in the lead up to the revolution",
        "criticism": "I need to make sure that I am not overlooking any important events or factors that contributed to the French Revolution. I should also be careful not to get bogged down in too much detail and lose sight of the big picture.",
        "speak": "I will start by researching the key events that led to the French Revolution and organizing them chronologically."
    },
    "command": {
        "name": "search",
        "args": {
            "tool_input": "Key events that led to the French Revolution"
        }
    }
}
System: Command search returned: Table of Contents · Causes of the French Revolution · Estates General · Rise of the Third Estate · Tennis Court Oath · The Bastille · Declaration of the Rights of Man ... Some key moments in the French Revolution, 1789- 1794; April 25. The first use of guillotine; June 13. Prussia declares war on France; August 9. Paris Commune ... 14 July 1789: Storming of Bastille ... A wave of revolutionary hysteria sweeps through Paris, leading to fears of a military coup. In retaliation, rebels storm ... 4 days ago ... Why did the French Revolution lead to war with other nations? ... distinguish that event from the later French revolutions of 1830 and 1848. Jun 18, 2021 ... In 1789, the French people were not happy. Due to war and King Louis XVI's spending habits, the country was utterly broke. On top of that, the ... Jan 24, 2019 ... Iran's Islamic Revolution shook the world in 1979, with currents that ... together a timeline featuring key events that led to the fall of ... A summary of the causes of the French Revolution, including the enlightenment, estates system, American Revolution, and absolutism. Eleven Major Events In the French Revolution · Call of the Estate General (May 5, 1789) · Tennis Court Oath (June 20, 1789) · Storming of the Bastille (July 14, ... Congress, led by Representative James Madison, enacts the first protective tariff. ... When the French Revolution turned violent in 1792, however, ... Dec 5, 2018 ... The execution of King Louis XVI and his wife Marie Antoinette resulted in the Revolution entering its most bloody stage known as the Reign of ...
Human: Determine which next command to use, and respond using the format specified above:

***System: You are Jim, Assistant Your decisions must always be made independently without seeking user assistance. Play to your strengths as an LLM and pursue simple strategies with no legal complications. If you have completed all your tasks, make sure to use the "finish" command.***

***GOALS:***

[…]

***Constraints:***

[…]

***Commands:***

[…]

***Resources:***

[…]

***Performance Evaluation:***

[…]

***You should only respond in JSON format as described below Response Format:*** 

[…]

***Ensure the response can be parsed by Python json.loads***

***System: The current time and date is Thu May 18 18:18:13 2023***

***System: This reminds you of these events from your past: ['Assistant Reply: {\n "thoughts": {\n "text": "I need to provide an analysis of the major historical events that led to the French Revolution. I should start by researching the key events that led to the revolution and organizing them chronologically.",\n "reasoning": "I need to have a clear understanding of the events that led to the French Revolution before I can provide an analysis. Organizing the events chronologically will help me to better understand the sequence of events and how they contributed to the revolution.",\n "plan": "- Research the key events that led to the French Revolution\n- Organize the events chronologically\n- Analyze the events to determine their significance in the lead up to the revolution",\n "criticism": "I need to make sure that I am not overlooking any important events or factors that contributed to the French Revolution. I should also be careful not to get bogged down in too much detail and lose sight of the big picture.",\n "speak": "I will start by researching the key events that led to the French Revolution and organizing them chronologically."\n },\n "command": {\n "name": "search",\n "args": {\n "tool_input": "Key events that led to the French Revolution"\n }\n }\n} \nResult: Command search returned: Table of Contents · Causes of the French Revolution · Estates General · Rise of the Third Estate · Tennis Court Oath · The Bastille · Declaration of the Rights of Man\xa0... Some key moments in the French Revolution, 1789- 1794; April 25. The first use of guillotine; June 13. Prussia declares war on France; August 9. Paris Commune\xa0... 14 July 1789: Storming of Bastille ... A wave of revolutionary hysteria sweeps through Paris, leading to fears of a military coup. In retaliation, rebels storm\xa0... 4 days ago ... Why did the French Revolution lead to war with other nations? ... distinguish that event from the later French revolutions of 1830 and 1848. Jun 18, 2021 ... In 1789, the French people were not happy. Due to war and King Louis XVI's spending habits, the country was utterly broke. On top of that, the\xa0... Jan 24, 2019 ... Iran's Islamic Revolution shook the world in 1979, with currents that ... together a timeline featuring key events that led to the fall of\xa0... A summary of the causes of the French Revolution, including the Enlightenment, the estates system, the American Revolution, and absolutism. Eleven Major Events In the French Revolution · Call of the Estate General (May 5, 1789) · Tennis Court Oath (June 20, 1789) · Storming of the Bastille (July 14,\xa0... Congress, led by Representative James Madison, enacts the first protective tariff. ... When the French Revolution turned violent in 1792, however,\xa0... Dec 5, 2018 ... The execution of King Louis XVI and his wife Marie Antoinette resulted in the Revolution entering its most bloody stage known as the Reign of\xa0... ']***

***Human: Determine which next command to use, and respond using the format specified above:*** 

***AI: {***

***"thoughts": {***

 ***"text": "I need to provide an analysis of the major historical events that led to the French Revolution. I should start by researching the key events that led to the revolution and organizing them chronologically.",***

***"reasoning": "I need to have a clear understanding of the events that led to the French Revolution before I can provide an analysis. Organizing the events chronologically will help me to better understand the sequence of events and how they contributed to the revolution.",***

***"plan": "- Research the key events that led to the French Revolution\n- Organize the events chronologically\n- Analyze the events to determine their significance in the lead up to the revolution",***

***"criticism": "I need to make sure that I am not overlooking any important events or factors that contributed to the French Revolution. I should also be careful not to get bogged down in too much detail and lose sight of the big picture.",***

***"speak": "I will start by researching the key events that led to the French Revolution and organizing them chronologically." },***

***"command": {*** 

***"name": "search",***

***"args": { "tool_input": "Key events that led to the French Revolution" }*** 

***}***

 ***}***

***System: Command search returned: Table of Contents · Causes of the French Revolution · Estates General · Rise of the Third Estate · Tennis Court Oath · The Bastille · Declaration of the Rights of Man ... Some key moments in the French Revolution, 1789- 1794; April 25. The first use of guillotine; June 13. Prussia declares war on France; August 9. Paris Commune ... 14 July 1789: Storming of Bastille ... A wave of revolutionary hysteria sweeps through Paris, leading to fears of a military coup. In retaliation, rebels storm ... 4 days ago ... Why did the French Revolution lead to war with other nations? ... distinguish that event from the later French revolutions of 1830 and 1848. Jun 18, 2021 ... In 1789, the French people were not happy. Due to war and King Louis XVI's spending habits, the country was utterly broke. On top of that, the ... Jan 24, 2019 ... Iran's Islamic Revolution shook the world in 1979, with currents that ... together a timeline featuring key events that led to the fall of ... A summary of the causes of the French Revolution, including the enlightenment, estates system, American Revolution, and absolutism. Eleven Major Events In the French Revolution · Call of the Estate General (May 5, 1789) · Tennis Court Oath (June 20, 1789) · Storming of the Bastille (July 14, ... Congress, led by Representative James Madison, enacts the first protective tariff. ... When the French Revolution turned violent in 1792, however, ... Dec 5, 2018 ... The execution of King Louis XVI and his wife Marie Antoinette resulted in the Revolution entering its most bloody stage known as the Reign of ...***

***Human: Determine which next command to use, and respond using the format specified above:***

In the line starting with “System: This reminds you of these events from your past:,” this time we see a non-empty list containing the LLM output from the previous interaction with the assistant, whose input and output are then added after the line “Human: Determine which next command to use and respond using the format specified above:” in a format similar to a few-shot example.

The complete AutoGPT execution with this goal lasted for several minutes, and we won’t show all its intermediate steps here. The last prompt inputted to the assistant contains the following:

|***AI: {***

***"thoughts": {***

***"text": "Now that I have summarized the major historical events that led to the French Revolution, I should analyze their significance and draw some conclusions about the revolution's impact on world history. I should start by organizing my thoughts and outlining my analysis.",***

***"reasoning": "Organizing my thoughts and outlining my analysis will help me to stay focused and ensure that I cover all the key points in my analysis. It will also help me to identify any gaps in my knowledge or areas where I need to do further research.",***

***"plan": "- Organize my thoughts and outline my analysis of the significance of the French Revolution\n- Identify any gaps in my knowledge or areas where I need to do further research",***

***"criticism": "I need to make sure that my analysis is well-supported by evidence and that I am accurately representing the historical events that led to the French Revolution. I should also be careful not to oversimplify the complex factors that contributed to the outbreak of the revolution.",***

***"speak": "I will organize my thoughts and outline my analysis of the significance of the French Revolution, and identify any gaps in my knowledge or areas where I need to do further research."***

***},***

***"command": {***

***"name": "write_file",***

***"args": {***

***"file_path": "french_revolution_analysis.txt",***

***"text": "Analysis of the Significance of the French Revolution:\\\\n\\\\nThe French Revolution was a watershed moment in world history that had a profound impact on the political, social, and economic landscape of Europe and beyond. Some of the key factors that contributed to the significance of the French Revolution include:\\\\n\\\\n- The rise of nationalism: The French Revolution helped to popularize the idea of nationalism, which would go on to play a major role in the political and social movements of the 19th and 20th centuries.\\\\n- The spread of liberal democracy: The French Revolution helped to spread the idea of liberal democracy, which would become the dominant political system in the Western world.\\\\n- The end of feudalism: The French Revolution marked the end of the feudal system in France and paved the way for the rise of capitalism and industrialization.\\\\n- The impact on international relations: The French Revolution had a major impact on international relations, leading to a series of wars and conflicts that would shape the course of European history for decades to come.\\\\ n\\\\n In conclusion, the French Revolution was a pivotal moment in world history that had a profound impact on the political, social, and economic landscape of Europe and beyond. By analyzing the significance of the French Revolution, we can gain a better understanding of the forces that have shaped the modern world.",***

***"append": false***

***}***

 ***}***

 ***}***

***System: Command write_file returned: File written successfully to french_revolution_analysis.txt. Human: Determine which next command to use, and respond using the format specified above:***

This prompt shows that the last executed command wrote the final overview into the “french_revolution_analysis.txt” file. Then, the assistant produced its final output.

I have completed my analysis of the significance of the French Revolution and have written it into a file. I will review my work and make any necessary revisions, and consider how to present my analysis to others.

System: You are Jim, Assistant
Your decisions must always be made independently without seeking user assistance.
Play to your strengths as an LLM and pursue simple strategies with no legal complications.
If you have completed all your tasks, make sure to use the "finish" command.

GOALS:

1. Provide an analysis of the major historical events that led to the French Revolution


Constraints:
1. ~4000-word limit for short-term memory. Your short-term memory is short, so immediately save important information to files.
2. If you are unsure how you previously did something or want to recall past events, thinking about similar events will help you remember.
3. No user assistance
4. Exclusively use the commands listed in double quotes e.g. "command name"

Commands:
1. search: Useful for when you need to answer questions about current events. You should ask targeted questions, args json schema: {"tool_input": {"type": "string"}}
2. write_file: Write the file to disk, args json schema: {"file_path": {"title": "File Path", "description": "name of file", "type": "string"}, "text": {"title": "Text", "description": "text to write to file", "type": "string"}, "append": {"title": "Append", "description": "Whether to append to an existing file.", "default": false, "type": "boolean"}}
3. read_file: Read the file from disk, args json schema: {"file_path": {"title": "File Path", "description": "name of file", "type": "string"}}
4. finish: use this to signal that you have finished all your objectives, args: "response": "final response to let people know you have finished your objectives"

Resources:
1. Internet access for searches and information gathering.
2. Long Term memory management.
3. GPT-3.5 powered Agents for delegation of simple tasks.
4. File output.

Performance Evaluation:
1. Continuously review and analyze your actions to ensure you are performing to the best of your abilities.
2. Constructively self-criticize your big-picture behavior constantly.
3. Reflect on past decisions and strategies to refine your approach.
4. Every command has a cost, so be smart and efficient. Aim to complete tasks in the least number of steps.

You should only respond in JSON format as described below
Response Format:
{
    "thoughts": {
        "text": "thought",
        "reasoning": "reasoning",
        "plan": "- short bulleted\n- list that conveys\n- long-term plan",
        "criticism": "constructive self-criticism",
        "speak": "thoughts summary to say to the user"
    },
    "command": {
        "name": "command name",
        "args": {
            "arg name": "value"
        }
    }
}
Ensure the response can be parsed by Python json.loads
System: The current time and date is Thu May 18 18:18:13 2023
System: This reminds you of these events from your past:
['Assistant Reply: {\n    "thoughts": {\n        "text": "I need to provide an analysis of the major historical events that led to the French Revolution. I should start by researching the key events that led to the revolution and organizing them chronologically.",\n        "reasoning": "I need to have a clear understanding of the events that led to the French Revolution before I can provide an analysis. Organizing the events chronologically will help me to better understand the sequence of events and how they contributed to the revolution.",\n        "plan": "- Research the key events that led to the French Revolution\\n- Organize the events chronologically\\n- Analyze the events to determine their significance in the lead up to the revolution",\n        "criticism": "I need to make sure that I am not overlooking any important events or factors that contributed to the French Revolution. I should also be careful not to get bogged down in too much detail and lose sight of the big picture.",\n        "speak": "I will start by researching the key events that led to the French Revolution and organizing them chronologically."\n    },\n    "command": {\n        "name": "search",\n        "args": {\n            "tool_input": "Key events that led to the French Revolution"\n        }\n    }\n} \nResult: Command search returned: Table of Contents · Causes of the French Revolution · Estates General · Rise of the Third Estate · Tennis Court Oath · The Bastille · Declaration of the Rights of Man\xa0... Some key moments in the French Revolution, 1789- 1794; April 25. The first use of guillotine; June 13. Prussia declares war on France; August 9. Paris Commune\xa0... 14 July 1789: Storming of Bastille ... A wave of revolutionary hysteria sweeps through Paris, leading to fears of a military coup. In retaliation, rebels storm\xa0... 4 days ago ... Why did the French Revolution lead to war with other nations? ... distinguish that event from the later French revolutions of 1830 and 1848. Jun 18, 2021 ... In 1789, the French people were not happy. Due to war and King Louis XVI\'s spending habits, the country was utterly broke. On top of that, the\xa0... Jan 24, 2019 ... Iran\'s Islamic Revolution shook the world in 1979, with currents that ... together a timeline featuring key events that led to the fall of\xa0... A summary of the causes of the French Revolution, including the Enlightenment, the estates system, the American Revolution, and absolutism. Eleven Major Events In the French Revolution · Call of the Estate General (May 5, 1789) · Tennis Court Oath (June 20, 1789) · Storming of the Bastille (July 14,\xa0... Congress, led by Representative James Madison, enacts the first protective tariff. ... When the French Revolution turned violent in 1792, however,\xa0... Dec 5, 2018 ... The execution of King Louis XVI and his wife Marie Antoinette resulted in the Revolution entering its most bloody stage known as the Reign of\xa0... ']


Human: Determine which next command to use, and respond using the format specified above:
AI: {
    "thoughts": {
        "text": "I need to provide an analysis of the major historical events that led to the French Revolution. I should start by researching the key events that led to the revolution and organizing them chronologically.",
        "reasoning": "I need to have a clear understanding of the events that led to the French Revolution before I can provide an analysis. Organizing the events chronologically will help me to better understand the sequence of events and how they contributed to the revolution.",
        "plan": "- Research the key events that led to the French Revolution\n- Organize the events chronologically\n- Analyze the events to determine their significance in the lead up to the revolution",
        "criticism": "I need to make sure that I am not overlooking any important events or factors that contributed to the French Revolution. I should also be careful not to get bogged down in too much detail and lose sight of the big picture.",
        "speak": "I will start by researching the key events that led to the French Revolution and organizing them chronologically."
    },
    "command": {
        "name": "search",
        "args": {
            "tool_input": "Key events that led to the French Revolution"
        }
    }
}
System: Command search returned: Table of Contents · Causes of the French Revolution · Estates General · Rise of the Third Estate · Tennis Court Oath · The Bastille · Declaration of the Rights of Man ... Some key moments in the French Revolution, 1789- 1794; April 25. The first use of guillotine; June 13. Prussia declares war on France; August 9. Paris Commune ... 14 July 1789: Storming of Bastille ... A wave of revolutionary hysteria sweeps through Paris, leading to fears of a military coup. In retaliation, rebels storm ... 4 days ago ... Why did the French Revolution lead to war with other nations? ... distinguish that event from the later French revolutions of 1830 and 1848. Jun 18, 2021 ... In 1789, the French people were not happy. Due to war and King Louis XVI's spending habits, the country was utterly broke. On top of that, the ... Jan 24, 2019 ... Iran's Islamic Revolution shook the world in 1979, with currents that ... together a timeline featuring key events that led to the fall of ... A summary of the causes of the French Revolution, including the enlightenment, estates system, American Revolution, and absolutism. Eleven Major Events In the French Revolution · Call of the Estate General (May 5, 1789) · Tennis Court Oath (June 20, 1789) · Storming of the Bastille (July 14, ... Congress, led by Representative James Madison, enacts the first protective tariff. ... When the French Revolution turned violent in 1792, however, ... Dec 5, 2018 ... The execution of King Louis XVI and his wife Marie Antoinette resulted in the Revolution entering its most bloody stage known as the Reign of ...
Human: Determine which next command to use, and respond using the format specified above:

System: You are Jim, Assistant
Your decisions must always be made independently without seeking user assistance.
Play to your strengths as an LLM and pursue simple strategies with no legal complications.
If you have completed all your tasks, make sure to use the "finish" command.

GOALS:

1. Provide an analysis of the major historical events that led to the French Revolution


Constraints:
1. ~4000-word limit for short-term memory. Your short-term memory is short, so immediately save important information to files.
2. If you are unsure how you previously did something or want to recall past events, thinking about similar events will help you remember.
3. No user assistance
4. Exclusively use the commands listed in double quotes e.g. "command name"

Commands:
1. search: Useful for when you need to answer questions about current events. You should ask targeted questions, args json schema: {"tool_input": {"type": "string"}}
2. write_file: Write the file to disk, args json schema: {"file_path": {"title": "File Path", "description": "name of file", "type": "string"}, "text": {"title": "Text", "description": "text to write to file", "type": "string"}, "append": {"title": "Append", "description": "Whether to append to an existing file.", "default": false, "type": "boolean"}}
3. read_file: Read the file from disk, args json schema: {"file_path": {"title": "File Path", "description": "name of file", "type": "string"}}
4. finish: use this to signal that you have finished all your objectives, args: "response": "final response to let people know you have finished your objectives"

Resources:
1. Internet access for searches and information gathering.
2. Long Term memory management.
3. GPT-3.5 powered Agents for delegation of simple tasks.
4. File output.

Performance Evaluation:
1. Continuously review and analyze your actions to ensure you are performing to the best of your abilities.
2. Constructively self-criticize your big-picture behavior constantly.
3. Reflect on past decisions and strategies to refine your approach.
4. Every command has a cost, so be smart and efficient. Aim to complete tasks in the least number of steps.

You should only respond in JSON format as described below
Response Format:
{
    "thoughts": {
        "text": "thought",
        "reasoning": "reasoning",
        "plan": "- short bulleted\n- list that conveys\n- long-term plan",
        "criticism": "constructive self-criticism",
        "speak": "thoughts summary to say to the user"
    },
    "command": {
        "name": "command name",
        "args": {
            "arg name": "value"
        }
    }
}
Ensure the response can be parsed by Python json.loads
System: The current time and date is Thu May 18 18:18:13 2023
System: This reminds you of these events from your past:
['Assistant Reply: {\n    "thoughts": {\n        "text": "I need to provide an analysis of the major historical events that led to the French Revolution. I should start by researching the key events that led to the revolution and organizing them chronologically.",\n        "reasoning": "I need to have a clear understanding of the events that led to the French Revolution before I can provide an analysis. Organizing the events chronologically will help me to better understand the sequence of events and how they contributed to the revolution.",\n        "plan": "- Research the key events that led to the French Revolution\\n- Organize the events chronologically\\n- Analyze the events to determine their significance in the lead up to the revolution",\n        "criticism": "I need to make sure that I am not overlooking any important events or factors that contributed to the French Revolution. I should also be careful not to get bogged down in too much detail and lose sight of the big picture.",\n        "speak": "I will start by researching the key events that led to the French Revolution and organizing them chronologically."\n    },\n    "command": {\n        "name": "search",\n        "args": {\n            "tool_input": "Key events that led to the French Revolution"\n        }\n    }\n} \nResult: Command search returned: Table of Contents · Causes of the French Revolution · Estates General · Rise of the Third Estate · Tennis Court Oath · The Bastille · Declaration of the Rights of Man\xa0... Some key moments in the French Revolution, 1789- 1794; April 25. The first use of guillotine; June 13. Prussia declares war on France; August 9. Paris Commune\xa0... 14 July 1789: Storming of Bastille ... A wave of revolutionary hysteria sweeps through Paris, leading to fears of a military coup. In retaliation, rebels storm\xa0... 4 days ago ... Why did the French Revolution lead to war with other nations? ... distinguish that event from the later French revolutions of 1830 and 1848. Jun 18, 2021 ... In 1789, the French people were not happy. Due to war and King Louis XVI\'s spending habits, the country was utterly broke. On top of that, the\xa0... Jan 24, 2019 ... Iran\'s Islamic Revolution shook the world in 1979, with currents that ... together a timeline featuring key events that led to the fall of\xa0... A summary of the causes of the French Revolution, including the Enlightenment, the estates system, the American Revolution, and absolutism. Eleven Major Events In the French Revolution · Call of the Estate General (May 5, 1789) · Tennis Court Oath (June 20, 1789) · Storming of the Bastille (July 14,\xa0... Congress, led by Representative James Madison, enacts the first protective tariff. ... When the French Revolution turned violent in 1792, however,\xa0... Dec 5, 2018 ... The execution of King Louis XVI and his wife Marie Antoinette resulted in the Revolution entering its most bloody stage known as the Reign of\xa0... ']


Human: Determine which next command to use, and respond using the format specified above:
AI: {
    "thoughts": {
        "text": "I need to provide an analysis of the major historical events that led to the French Revolution. I should start by researching the key events that led to the revolution and organizing them chronologically.",
        "reasoning": "I need to have a clear understanding of the events that led to the French Revolution before I can provide an analysis. Organizing the events chronologically will help me to better understand the sequence of events and how they contributed to the revolution.",
        "plan": "- Research the key events that led to the French Revolution\n- Organize the events chronologically\n- Analyze the events to determine their significance in the lead up to the revolution",
        "criticism": "I need to make sure that I am not overlooking any important events or factors that contributed to the French Revolution. I should also be careful not to get bogged down in too much detail and lose sight of the big picture.",
        "speak": "I will start by researching the key events that led to the French Revolution and organizing them chronologically."
    },
    "command": {
        "name": "search",
        "args": {
            "tool_input": "Key events that led to the French Revolution"
        }
    }
}
System: Command search returned: Table of Contents · Causes of the French Revolution · Estates General · Rise of the Third Estate · Tennis Court Oath · The Bastille · Declaration of the Rights of Man ... Some key moments in the French Revolution, 1789- 1794; April 25. The first use of guillotine; June 13. Prussia declares war on France ; August 9. Paris Commune ... 14 July 1789: Storming of Bastille ... A wave of revolutionary hysteria sweeps through Paris, leading to fears of a military coup. In retaliation, rebels storm ... 4 days ago ... Why did the French Revolution lead to war with other nations? ... distinguish that event from the later French revolutions of 1830 and 1848. Jun 18, 2021 ... In 1789, the French people were not happy. Due to war and King Louis XVI's spending habits, the country was utterly broke. On top of that, the ... Jan 24, 2019 ... Iran's Islamic Revolution shook the world in 1979, with currents that ... together a timeline featuring key events that led to the fall of ... A summary of the causes of the French Revolution, including the enlightenment, estates system, American Revolution, and absolutism. Eleven Major Events In the French Revolution · Call of the Estate General (May 5, 1789) · Tennis Court Oath (June 20, 1789) · Storming of the Bastille (July 14, ... Congress, led by Representative James Madison, enacts the first protective tariff. ... When the French Revolution turned violent in 1792, however, ... Dec 5, 2018 ... The execution of King Louis XVI and his wife Marie Antoinette resulted in the Revolution entering its most bloody stage known as the Reign of ...
Human: Determine which next command to use, and respond using the format specified above:

***Analysis of the Significance of the French Revolution:***

***The French Revolution was a watershed moment in world history that had a profound impact on the political, social, and economic landscape of Europe and beyond. Some of the key factors that contributed to the significance of the French Revolution include:***

- ***The rise of nationalism: The French Revolution helped to popularize the idea of nationalism, which would go on to play a major role in the political and social movements of the 19th and 20th centuries.***
- ***The spread of liberal democracy: The French Revolution helped to spread the idea of liberal democracy, which would become the dominant political system in the Western world.***
- ***The end of feudalism: The French Revolution marked the end of the feudal system in France and paved the way for the rise of capitalism and industrialization.***
- ***The impact on international relations: The French Revolution had a major impact on international relations, leading to a series of wars and conflicts that would shape the course of European history for decades to come.***

***In conclusion, the French Revolution was a pivotal moment in world history that had a profound impact on the political, social, and economic landscape of Europe and beyond. By analyzing the significance of the French Revolution, we can gain a better understanding of the forces that have shaped the modern world.***

As we can see, based on the output Jim, the AI assistant, has performed well, managing to work on three distinct and detailed files. 

Jim’s performance highlights several key capabilities:

Research and Analysis: The assistant has demonstrated good research and analysis skills. It has managed to delve into the historical context, key events, and the long-term impact of the French Revolution, presenting them in an organized and comprehensible manner.
Writing and Summarization: The agent displayed good writing skills. It has effectively synthesized complex historical concepts into clear and concise summaries, enabling better understanding for any reader, irrespective of their background knowledge.
Planning and Workflow Management: It has shown a structured approach in managing the tasks. It completed its research, wrote summaries, and planned for review and presentation, all while maintaining a smooth workflow and ensuring the proper organization and storage of information.
Autonomy: We are able to see that Jim operated independently, without any user assistance, demonstrating their ability to handle tasks from beginning to end.
The results showcase its capabilities in historical research, analysis, summarization, planning, and information management.

### Conclusion
Wrapping things up, the AI assistant we've been discussing makes great use of a collection of tools, including a search function and tools to read and write files. These tools allow the assistant to dig deep into historical topics, pull apart the complex factors at play, and manage data effectively.

The assistant's ability to dive deep into topics, strategize effectively, and communicate its findings clearly, truly highlights the potential of AutoGPT-based applications.

In the next lesson, we’ll learn about agent simulations projects, like CAMEL and Generative Agents.